In [1]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
sns.set()

In [3]:
df = pd.read_csv('../data.csv')

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_11647/3454644713.py:1: DtypeWarning: Columns (3,4,7,8,11,12,14,15,16,19,45,46,49,56,69,83,95,98,99,100,104,108,109,119,120,121,122,123,124,125,127,128,129,141,143,144,145,146,147,148,155,188,217,218,219,220,221,222,223,224,226,229,232,233,234,235,236,237,238,239,240,243,244,245,246,248,249,250,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


In [4]:
df = df[df['psic'].notna()] #drop 45 NaN participant SIC, fix the issue by not solving it
telDF = df[df['psicp'].str.contains('481')] #Create df who have at least one participant in the tellcom industry with SIC: https://siccode.com/sic-code/481/telephone-communications
telDF = telDF.drop('Unnamed: 0', axis=1)
telDF = telDF.reset_index(drop=True)

#Export
telDF.to_csv('teldf.csv', index=False) #create csv sample

### Analyse companies in DF